## Interactive Shell

In [ ]:
from data_utils import get_trimmed_glove_vectors, load_vocab, \
    get_processing_word, Data
from config import Config
from cnn_lstm_crf import CnnLstmCrfModel

config = Config()

vocab_words = load_vocab(config.words_filename)
vocab_mor_tags = load_vocab(config.mor_tags_filename)
vocab_tags  = load_vocab(config.tags_filename)
vocab_chars = load_vocab(config.chars_filename)
vocab_lex_tags = load_vocab(config.lex_tags_filename)

# get processing functions
processing_word = get_processing_word(vocab_words, vocab_chars,
                lowercase=True, chars=config.chars)
processing_mor_tag = get_processing_word(vocab_mor_tags, lowercase=False)
processing_lex_tag = get_processing_word(vocab_lex_tags, lowercase=False)


# get pre trained embeddings
embeddings = get_trimmed_glove_vectors(config.trimmed_filename)

cnn_model = CnnLstmCrfModel(config, embeddings, ntags=len(vocab_tags),nchars=len(vocab_chars))
cnn_model.build()

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
cnn_model.interactive_shell(vocab_tags, processing_word, processing_mor_tag, processing_lex_tag)

INFO:tensorflow:Restoring parameters from data/model/model_fil_2345_600_100_128_drop_0.8_0924.weights


Restoring parameters from data/model/model_fil_2345_600_100_128_drop_0.8_0924.weights

            This is an interactive mode.
            To exit, enter 'exit'.
            


input>  국내 영화시장 규모는 최근 3년간 2조 원대에서 정체된 상태인 해외 로컬 제작 방식으로 만든 '수상한 그녀'가 중국, 베트남, 일본, 태국 등 5개국 리메이크돼 총 780억 원의 매출로 성공을 거둔 바 있습니다. 
output> 국내 영화시장 규모는 최근 <3년:DT>간 2조 원대에서 정체된 상태인 해외 로컬 제작 방식으로 만든 수상한 그녀가 <중국:LC>, <베트남:LC>, <일본:LC>, <태국:LC> 등 5개국 리메이크돼 총 780억 원의 매출로 성공을 거둔 바 있습니다.

3년	DT

중국	LC

베트남	LC

일본	LC

태국	LC



## Write File

In [1]:
from data_utils import get_trimmed_glove_vectors, load_vocab, \
    get_processing_word, Data
from config import Config
from cnn_lstm_crf import CnnLstmCrfModel

config = Config()

vocab_words = load_vocab(config.words_filename)
vocab_mor_tags = load_vocab(config.mor_tags_filename)
vocab_tags  = load_vocab(config.tags_filename)
vocab_chars = load_vocab(config.chars_filename)
vocab_lex_tags = load_vocab(config.lex_tags_filename)

# get processing functions
processing_word = get_processing_word(vocab_words, vocab_chars,
                lowercase=True, chars=config.chars)
processing_mor_tag = get_processing_word(vocab_mor_tags, lowercase=False)
processing_lex_tag = get_processing_word(vocab_lex_tags, lowercase=False)


# get pre trained embeddings
embeddings = get_trimmed_glove_vectors(config.trimmed_filename)

cnn_model = CnnLstmCrfModel(config, embeddings, ntags=len(vocab_tags),nchars=len(vocab_chars))
cnn_model.build()

cnn_model.write_tag_result_test(vocab_tags, processing_word, processing_mor_tag, processing_lex_tag)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from data/model/model_fil_2345_600_100_128_drop_0.8_0924.weights


Restoring parameters from data/model/model_fil_2345_600_100_128_drop_0.8_0924.weights
